필요한 라이브러리 불러오기


In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

from xgboost.sklearn import XGBClassifier,XGBRegressor
from lightgbm.sklearn import LGBMClassifier,LGBMRegressor

from sklearn.model_selection import  KFold
from functools import partial

import optuna


데이터 불러오기

In [37]:
df_tr= pd.read_excel('C:/Users/jeong bok/Documents/cse101/team9PJ/mlops-team9/data/1월 서울시 교통량 조사자료.xlsx',sheet_name=1)


df_air = pd.read_csv('C:/Users/jeong bok/Documents/cse101/team9PJ/mlops-team9/data/new_AIR_HOUR_2018.csv')

In [38]:
df_tr.head(5)

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,20180101,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,657.0,617.0,405.0,279.0,...,1401.0,1592.0,1515.0,1447.0,1192.0,1037.0,918.0,881.0,819.0,533.0
1,20180102,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,382.0,256.0,166.0,171.0,...,1690.0,1835.0,1875.0,2021.0,2015.0,1612.0,1236.0,1308.0,1209.0,951.0
2,20180103,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,641.0,492.0,310.0,238.0,...,1841.0,2007.0,1940.0,2096.0,2209.0,1653.0,1387.0,1295.0,1248.0,956.0
3,20180104,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,661.0,483.0,357.0,279.0,...,1890.0,2030.0,2058.0,2123.0,2113.0,1619.0,1327.0,1379.0,1318.0,984.0
4,20180105,금,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,764.0,524.0,361.0,282.0,...,1860.0,2134.0,2163.0,2282.0,2307.0,1723.0,1577.0,1460.0,1371.0,1102.0


In [39]:
df_air.head(5)

,Unnamed: 0,날짜,시간,구,PM10,PM2.5,O3,SO2,NO2,CO
0,0,2018-01-01,0,종로구,29.0,17.0,0.022,0.006,0.021,0.4
1,1,2018-01-01,0,중구,34.0,19.0,0.020,0.004,0.020,0.5
2,2,2018-01-01,0,용산구,29.0,14.0,0.013,0.004,0.023,0.3
3,3,2018-01-01,0,은평구,43.0,25.0,0.017,0.003,0.018,0.6
4,4,2018-01-01,0,서대문구,0.0,0.0,0.000,0.000,0.000,0.0


데이터 전처리를 진행합니다

In [31]:
melted_df = pd.melt(df_tr, id_vars=['일자', '요일', '지점명', '지점번호', '구분', '방향'],
                    var_name='시간', value_name='값')

In [32]:
melted_df.head(5)

,일자,요일,지점명,지점번호,구분,방향,시간,값
0,20180101,월,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,657.0
1,20180102,화,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,382.0
2,20180103,수,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,641.0
3,20180104,목,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,661.0
4,20180105,금,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,764.0


In [43]:
melted_df.reset_index(drop=True, inplace=True)


In [44]:
melted_df

,일자,요일,지점명,지점번호,구분,방향,시간,값
0,20180101,월,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,657.0
1,20180102,화,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,382.0
2,20180103,수,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,641.0
3,20180104,목,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,661.0
4,20180105,금,성산로(금화터널),A-01,봉원고가차도->독립문역,유입,0시,764.0
...,...,...,...,...,...,...,...,...
200875,20180127,토,서부간선도로,F-09,목동교->신정교,유출,23시,2736.0
200876,20180128,일,서부간선도로,F-09,목동교->신정교,유출,23시,2278.0
200877,20180129,월,서부간선도로,F-09,목동교->신정교,유출,23시,2493.0
200878,20180130,화,서부간선도로,F-09,목동교->신정교,유출,23시,2185.0


In [45]:
df_air.reset_index(drop=True,inplace=True)

In [48]:
df_air.drop(df_air.columns[0], axis=1)

,날짜,시간,구,PM10,PM2.5,O3,SO2,NO2,CO
0,2018-01-01,0,종로구,29.0,17.0,0.022,0.006,0.021,0.4
1,2018-01-01,0,중구,34.0,19.0,0.020,0.004,0.020,0.5
2,2018-01-01,0,용산구,29.0,14.0,0.013,0.004,0.023,0.3
3,2018-01-01,0,은평구,43.0,25.0,0.017,0.003,0.018,0.6
4,2018-01-01,0,서대문구,0.0,0.0,0.000,0.000,0.000,0.0
...,...,...,...,...,...,...,...,...,...
264746,2018-12-31,23,관악구,36.0,24.0,0.004,0.005,0.059,0.6
264747,2018-12-31,23,서초구,36.0,13.0,0.002,0.004,0.052,0.5
264748,2018-12-31,23,강남구,35.0,26.0,0.002,0.005,0.044,0.7
264749,2018-12-31,23,송파구,43.0,27.0,0.002,0.004,0.057,0.9
